In [154]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import MeanShift, estimate_bandwidth

In [155]:
titanic_data = pd.read_excel('titanic.xls')
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [156]:
titanic_data.drop(['name','ticket','cabin', 'body','home.dest','boat'], 1 ,inplace=True)
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S
2,1,0,female,2.0000,1,2,151.5500,S
3,1,0,male,30.0000,1,2,151.5500,S
4,1,0,female,25.0000,1,2,151.5500,S


In [157]:
le = preprocessing.LabelEncoder()
titanic_data['sex'] = le.fit_transform(titanic_data["sex"].astype(str))
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,0,29.0000,0,0,211.3375,S
1,1,1,1,0.9167,1,2,151.5500,S
2,1,0,0,2.0000,1,2,151.5500,S
3,1,0,1,30.0000,1,2,151.5500,S
4,1,0,0,25.0000,1,2,151.5500,S


In [158]:
#using one hot encoding
titanic_data = pd.get_dummies(titanic_data,columns=['embarked'])
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,0,0,1
1,1,1,1,0.9167,1,2,151.5500,0,0,1
2,1,0,0,2.0000,1,2,151.5500,0,0,1
3,1,0,1,30.0000,1,2,151.5500,0,0,1
4,1,0,0,25.0000,1,2,151.5500,0,0,1


In [159]:
#check for missing data values
titanic_data[titanic_data.isnull().any(axis=1)]

#We will drop them.
titanic_data = titanic_data.dropna()

In [160]:
clf = MeanShift(bandwidth=30)
#We will provide only bandwith in hyperparameter . 
#The smaller values of bandwith result in tall skinny kernels 
#& larger values result in short fat kernels.
#We found the bandwith using the estimate_bandiwth function mentioned in below cell.

clf.fit(titanic_data)

MeanShift(bandwidth=30, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [161]:
estimate_bandwidth(titanic_data)

31.849187320670215

In [162]:
labels = clf.labels_
np.unique(labels)
#Thus a bandwith of 30 produces 5 clusters - 
#every point is assigned to one of these clusters.

[4 2 2 ... 0 0 0]


array([0, 1, 2, 3, 4, 5])

In [163]:
#We will add a new column in dataset which 
#shows the cluster the data of a particular row belongs to.
titanic_data['cluster_group'] = np.nan
data_length=len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i,titanic_data.columns.get_loc('cluster_group')] = labels[i]
titanic_data.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S,cluster_group
0,1,1,0,29.0000,0,0,211.3375,0,0,1,4.0
1,1,1,1,0.9167,1,2,151.5500,0,0,1,2.0
2,1,0,0,2.0000,1,2,151.5500,0,0,1,2.0
3,1,0,1,30.0000,1,2,151.5500,0,0,1,2.0
4,1,0,0,25.0000,1,2,151.5500,0,0,1,2.0


In [164]:
titanic_data.describe()

,pclass,survived,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S,cluster_group
count,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000
mean,2.206699,0.408612,0.628708,29.851834,0.503349,0.421053,36.686080,0.202871,0.047847,0.747368,0.365550
std,0.841542,0.491813,0.483382,14.389201,0.912471,0.840052,55.732533,0.402330,0.213544,0.434729,0.816708
min,1.000000,0.000000,0.000000,0.166700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,21.000000,0.000000,0.000000,8.050000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,1.000000,28.000000,0.000000,0.000000,15.750000,0.000000,0.000000,1.000000,0.000000
75%,3.000000,1.000000,1.000000,39.000000,1.000000,1.000000,35.500000,0.000000,0.000000,1.000000,0.000000
max,3.000000,1.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,5.000000


In [165]:
#Grouping passengers by Cluster
titanic_cluster_data = titanic_data.groupby(['cluster_group']).mean()
#Count of passengers in each cluster
titanic_cluster_data['Counts'] = pd.Series(titanic_data.groupby(['cluster_group']).size())
titanic_cluster_data


,pclass,survived,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,2.514963,0.339152,0.675810,27.883105,0.425187,0.346633,15.326999,0.130923,0.058603,0.810474,802
1.0,1.280488,0.597561,0.524390,36.746951,0.798780,0.554878,64.737935,0.365854,0.018293,0.603659,164
2.0,1.000000,0.720930,0.395349,33.881784,0.581395,0.744186,133.555521,0.558140,0.000000,0.441860,43
3.0,1.000000,0.750000,0.312500,35.062500,1.437500,1.937500,259.824212,0.625000,0.000000,0.375000,16
4.0,1.000000,0.625000,0.312500,39.031250,0.375000,0.375000,217.392975,0.562500,0.000000,0.437500,16
5.0,1.000000,1.000000,0.500000,41.000000,0.000000,0.500000,512.329200,1.000000,0.000000,0.000000,4
